In [200]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [178]:
hospital_data = pd.read_excel('Hospital_dataset.xlsx', sheet_name='Hospital ER')

In [179]:
hospital_data_copy = hospital_data.copy()

In [180]:
hospital_data_copy.head(10)

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,Full_Name,Doctor,patient_admin_flag,patient_waittime,department_referral,revenue
0,2020-03-20 08:47:00,145-39-5406,Male,69,10.0,H,Glasspool,HGlasspool,Dr. Ben Carson,False,39,NaN,260
1,2020-06-15 11:29:00,316-34-3057,Male,4,NaN,X,Methuen,XMethuen,Dr. Mohamed Oz,True,27,NaN,275
2,2020-06-20 09:13:00,897-46-3852,Female,56,9.0,P,Schubuser,PSchubuser,Dr. Patricia Bath,True,55,General Practice,260
3,2020-02-04 22:34:00,358-31-9711,Female,24,8.0,U,Titcombe,UTitcombe,Dr. Jennifer Doudna,True,31,General Practice,460
4,2020-09-04 17:48:00,289-26-0537,Male,5,NaN,Y,Gionettitti,YGionettitti,Dr. Joycelyn Elders,False,10,Orthopedics,275
5,2019-04-20 00:13:00,255-51-2877,Male,58,NaN,H,Buff,HBuff,Dr. Paul Farmer,False,59,NaN,260
6,2019-08-23 08:26:00,465-97-0990,Female,68,NaN,F,Perrat,FPerrat,Dr. Eric Topol,True,43,NaN,260
7,2019-07-29 16:57:00,157-31-7520,Female,47,NaN,K,Gwillim,KGwillim,Dr. Lisa Sanders,True,23,NaN,460
8,2020-02-19 06:54:00,432-34-5614,Female,79,1.0,E,Dewhirst,EDewhirst,Dr. Francis Collins,False,42,NaN,260
9,2020-10-11 05:25:00,609-17-8678,Male,62,NaN,M,Crebo,MCrebo,Dr. Joycelyn Elders,False,51,NaN,260


In [181]:
hospital_data_copy['patient_gender'] = hospital_data_copy['patient_gender'].astype('category')
hospital_data_copy['patient_first_inital'] = hospital_data_copy['patient_first_inital'].astype('category')
hospital_data_copy['Doctor'] = hospital_data_copy['Doctor'].astype('category')
hospital_data_copy['department_referral'] = hospital_data_copy['department_referral'].astype('category')

In [ ]:
hospital_data_copy['Full_Name'] = hospital_data_copy['Full_Name'].str.slice(0,1) +"."+ hospital_data_copy['Full_Name'].str.slice(1,)

0          H.Glasspool
1            X.Methuen
2          P.Schubuser
3           U.Titcombe
4        Y.Gionettitti
             ...      
9212           A.Crich
9213           A.Gerbl
9214          J.Dowall
9215          W.Rustan
9216    H.Hickeringill
Name: Full_Name, Length: 9217, dtype: object

In [183]:
hospital_data_copy['patient_sat_score'].fillna(hospital_data_copy['patient_sat_score'].median(), inplace=True)

In [197]:
labels = ['Child', 'Teen', 'Young Adult', 'Adult', 'Senior']
bins = [0, 13, 20, 36, 60, np.inf]
hospital_data_copy['patient_category'] = pd.cut(hospital_data_copy['patient_age'], bins=bins,labels=labels,)

In [205]:
hospital_data_copy.groupby("Doctor")['patient_category'].value_counts()

Doctor             patient_category
Dr. Anthony Fauci  Adult               165
                   Young Adult         113
                   Senior              106
                   Child                78
                   Teen                 41
                                      ... 
Dr. Sanjay Gupta   Adult               167
                   Senior              156
                   Young Adult         132
                   Child               112
                   Teen                 54
Name: count, Length: 65, dtype: int64

In [192]:
from imputing_method import imputer

In [185]:
grouped_by_doc = hospital_data_copy.groupby('Doctor')

In [186]:
hospital_data_copy['department_referral'] = hospital_data_copy['department_referral'] = hospital_data_copy.apply(
    lambda row: imputer(row, grouped_by_doc) if pd.isna(row['department_referral']) else row['department_referral'],
    axis=1
)

In [187]:
hospital_data_copy['department_referral'].value_counts()

department_referral
General Practice    1972
Renal               1574
Gastroenterology    1361
Orthopedics         1275
Physiotherapy       1242
Neurology           1116
Cardiology           677
Name: count, dtype: int64

In [188]:
print(hospital_data['department_referral'].value_counts(),"\n", hospital_data['department_referral'].isna().sum())

department_referral
General Practice    1840
Orthopedics          995
Physiotherapy        276
Cardiology           248
Neurology            193
Gastroenterology     178
Renal                 86
Name: count, dtype: int64 
 5401


In [189]:
hospital_data_copy.isna().sum()

date                    0
patient_id              0
patient_gender          0
patient_age             0
patient_sat_score       0
patient_first_inital    0
patient_last_name       0
Full_Name               0
Doctor                  0
patient_admin_flag      0
patient_waittime        0
department_referral     0
revenue                 0
dtype: int64